In [147]:
import pandas as pd
import numpy as np

import dateutil
import datetime
import time
from time import sleep

from bs4 import BeautifulSoup
import requests
import random
import re
import html5lib
import html
import pickle as pk
from fake_useragent import UserAgent
import json

In [148]:
import selenium

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from pyvirtualdisplay import Display

import os

In [149]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print f.__name__, 'took', end - start, 'seconds'
        return result
    return f_timer

In [14]:
!pwd

/Users/feiwang/Documents/python_code/Petition Prediction


In [139]:
# Setting up chromedriver
chromedriver = '/Users/feiwang/Documents/python_code/Petition Prediction/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
#chromedriver
driver = webdriver.Chrome(chromedriver)

In [26]:
# go straight to most-recent pages
petitions_url = 'https://www.change.org/petitions#most-recent/' # up to 5084

@timefunc
def get_petition_urls(base, start, stop):
    d = {}
    for i in range(start, stop+1):
        driver.get(base + str(i))
        petitions = driver.find_elements_by_xpath('//div[@class = "petition-list"]//ol//li[@class = "petition"]')
        for p in petitions:
            d[p.get_attribute('data-id')] = p.get_attribute('data-url')
    return d

@timefunc
def chunk_petition_urls(dic, base, first, last, chunk):
    for c in range(first, last+1, chunk):
        d = get_petition_urls(base, c, c+chunk)
        dic.update(d)
        filename = 'master' + str(c+chunk) + '.pkl'
        with open(filename, 'wb') as f:
            pk.dump(dic,f,-1)
    return dic

In [27]:
master = {}

In [28]:
urls1000 = chunk_petition_urls(master, petitions_url, 1, 1000, 100)

get_petition_urls took 449.227883101 seconds
get_petition_urls took 433.852669954 seconds
get_petition_urls took 406.804764986 seconds
get_petition_urls took 409.117487907 seconds
get_petition_urls took 335.363702059 seconds
get_petition_urls took 367.451252937 seconds
get_petition_urls took 443.21186614 seconds
get_petition_urls took 4.51107311249 seconds
get_petition_urls took 11.0591571331 seconds
get_petition_urls took 4.56889796257 seconds
chunk_petition_urls took 2865.644835 seconds


In [29]:
urls2000 = chunk_petition_urls(master, petitions_url, 1001, 2000, 100)

get_petition_urls took 128.451766968 seconds
get_petition_urls took 4.92390394211 seconds
get_petition_urls took 4.66733503342 seconds
get_petition_urls took 4.63202905655 seconds
get_petition_urls took 5.25774693489 seconds
get_petition_urls took 4.41870188713 seconds
get_petition_urls took 5.14406204224 seconds
get_petition_urls took 4.8712208271 seconds
get_petition_urls took 4.80128002167 seconds
get_petition_urls took 50.7472710609 seconds
chunk_petition_urls took 218.685530186 seconds


In [30]:
len(master)

7610

In [204]:
master
pk.dump(master, open('master_recent_url.pickle', 'wb'))

In [152]:
chromedriver = '/Users/feiwang/Documents/python_code/Petition Prediction/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
#chromedriver
driver = webdriver.Chrome(chromedriver)

victories_url = 'https://www.change.org/victories#most-recent/' 

@timefunc
def get_victory_urls(base, start, stop):
    d = {}
    for i in range(start, stop+1):
        driver.get(base + str(i))
        sleep(random.randint(1,3))
        victories = driver.find_elements_by_xpath('//div[@class = "petition-list"]//ol//li[@class = "petition"]')
        for i, v in enumerate(victories):
            d[i] = v.get_attribute('data-url')
    return d

@timefunc
def chunk_victory_urls(dic, base, first, last, chunk):
    for c in range(first, last+1, chunk):
        d = get_victory_urls(base, c, c+chunk)
        dic.update(d)
        filename = 'victories' + str(c+chunk) + '.pkl'
        with open(filename, 'wb') as f:
            pk.dump(dic,f,-1)
    return dic

In [192]:
victories = {}
victories = get_victory_urls(victories_url, 1, 200)
filename = 'victories_final.pkl'
with open(filename, 'wb') as f:
    pk.dump(victories,f,-1)

get_victory_urls took 988.683190823 seconds


In [209]:
str(master.values()[0])

'https://www.change.org/p/chico-state-petition-for-justice-hcsv-dept'

In [ ]:
@timefunc
def click_100_buttons(driver):
    try:
        load_more = driver.find_element_by_xpath('//button[@class="btn btn-full btn-big"]')
        for i in range(100):
            try:
                load_more.click()
                driver.execute_script("window.scrollTo(0,1000000000);")
                sleep(.1)
            except:
                break
    except:
        pass
    sleep(1)

In [154]:
chromedriver = '/Users/feiwang/Documents/python_code/Petition Prediction/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
#chromedriver
driver = webdriver.Chrome(chromedriver)

In [126]:
web_vic_dict=dict()
def get_petittion_info(url):    
    sleep(random.randint(1,2))
    one_dict=dict()
    driver.get(url)
    
    try:
        titles=driver.find_element_by_tag_name('h1').text
    except NoSuchElementException:
        return one_dict
    print titles
    
    try:
        total_supporter=driver.find_element_by_xpath \
        ('//div[@class = "col-xs-4 type-s js-mobile-supporter-count"]').text
    except NoSuchElementException:
        return one_dict   
    print total_supporter
    
    try:
        text=driver.find_element_by_xpath \
        ('//div[@class = "rte js-description-content"]').text
    except NoSuchElementException:
        one_dict[url]=[titles,total_supporter,[],[]]
        return one_dict
    
    sleep(random.randint(1,3))
    driver.execute_script("window.scrollTo(0,1000000000);")
    
    try:
        supporters_progression=driver.find_elements_by_xpath \
        ('//div[@class = "media mvn type-s type-highlight"]')
    except NoSuchElementException:
        one_dict[url]=[titles,text,total_supporter,[],[]]
        return one_dict
    sup_prog=[]
    for supporter in supporters_progression:
        sup_prog.append(supporter.text)

    try:
        starting_date=driver.find_element_by_xpath \
        ('//div[@class = "box box-basic bg-default"]').text
    except NoSuchElementException:
        starting_date = []
    print starting_date
    one_dict[url]=[titles,text,total_supporter,sup_prog,starting_date]
    
    return one_dict

    

In [88]:
get_petittion_info(vic_url[12])

Grant Clemency to Ricky G. Minor
154,309
Supporters
3 years ago
Heather Minor started this petition


{u'https://www.change.org/p/my-dad-will-die-in-prison-for-a-nonviolent-drug-offense': [u'Grant Clemency to Ricky G. Minor',
  u'My name is Heather Minor and on February 6, 2001, when I was 7 years old, my life took a horrible turn when my father was sentenced to life in prison for a nonviolent drug offense. Without clemency from President Obama, he will die in prison. \nWhen my father, Ricky Minor, was given his sentence I didn\'t realize I was about to begin a journey that would become a new "normal" way of life for me \u2013 growing up without a father. Even though my dad was a first time nonviolent drug offender, under mandatory minimum sentencing laws he was sentenced to life in prison without the possibility of parole. On that same day, my mother was also arrested. My whole world as I knew it was completely destroyed. At the age of seven, both of my parents were taken away from me, and from that point forward I was raised by my elderly grandparents.\nI am now 21 years old, the fir

In [136]:
def assemble_pettion_info(master_storage,url_dict,start_key):
    chunck_dict=dict()
    i=0
    print 'from'+ str(start_key)+ 'url'
    for k in xrange(start_key,len(url_dict)):
        one_dict=get_petittion_info(url_dict[k])
        chunck_dict.update(one_dict)
        master_storage.update(one_dict)
        i+=1
        print i
        
    pk.dump(chunck_dict, \
            open('vic_pettion_info_dict'+str(start_key)+'-'+str(start_key+i)+'_400.pk','wb'))
    pk.dump(master_storage, open('vic_pettion_info_master_dict.pk','wb'))    
    
    return chunck_dict

In [133]:
web_vic_dict=dict()


In [117]:
len(vic_url)

1990

[0, 1, 2, 3, 4]

In [118]:
with open('victories_url400_0609.pkl','rb') as f:
    vic_url400=pk.load(f)
len(vic_url400)

# with open('victories_url600_0609.pkl','rb') as f:
#     vic_url600=pk.load(f)
    
# with open('victories_url800_0609.pkl','rb') as f:
#     vic_url800=pk.load(f)

3680

In [128]:
web_vic_dict=dict()

In [134]:
chunck_dict200=assemble_pettion_info(web_vic_dict,vic_url,0)

from0url
Don't abandon chimpanzees for whom you promised to provide lifetime care!
218,723
Supporters
[]
1
Save expiring benefits for retired coal miners and their families.
55,176
Supporters
2 months ago
John Leach started this petition
2
Change the Rating of "3 Generations" to PG-13
34,750
Supporters
2 months ago
Blair Durkee started this petition
3
We Demand that the U. S. Senate not confirm Andrew Puzder as Secretary of Labor
37,644
Supporters
5 months ago
TRO Political Action Network Truth. Resistance. Opposition. started this petition
4
Make South Orange a Sanctuary City! Stand Up for Immigrants!
603
Supporters
4 months ago
SOMA Action started this petition
5
Exempt Wartime Interpreters from President Trump's Executive Order Banning Immigration
40,932
Supporters
4 months ago
No One Left Behind started this petition
6
Pardon all of the estimated 49,000 men who, like Alan Turing, were convicted of consenting same-sex relations under the British "gross indecency" law (only repealed 

In [138]:
chunck_dict400=assemble_pettion_info(web_vic_dict,vic_url400,0)

from0url
Don't abandon chimpanzees for whom you promised to provide lifetime care!
218,723
Supporters
2 years ago
Dr. Brian Hare started this petition
1
Save expiring benefits for retired coal miners and their families.
55,176
Supporters
2 months ago
John Leach started this petition
2
Change the Rating of "3 Generations" to PG-13
34,750
Supporters
2 months ago
Blair Durkee started this petition
3
We Demand that the U. S. Senate not confirm Andrew Puzder as Secretary of Labor
37,644
Supporters
5 months ago
TRO Political Action Network Truth. Resistance. Opposition. started this petition
4
Make South Orange a Sanctuary City! Stand Up for Immigrants!
603
Supporters
4 months ago
SOMA Action started this petition
5
Exempt Wartime Interpreters from President Trump's Executive Order Banning Immigration
40,932
Supporters
4 months ago
No One Left Behind started this petition
6
Pardon all of the estimated 49,000 men who, like Alan Turing, were convicted of consenting same-sex relations under the

In [113]:
len(web_vic_dict)

1573

In [114]:
pk.dump(web_vic_dict, open('vic_pettion_info_dict_705-1773.pk','wb'))

In [4]:
with open('victories_url200_0609.pkl','rb') as f:
    vic_url=pk.load(f)

In [388]:
clean_recent_dict={}
remove_fish=lambda x: re.sub('(<.*?>)'," ",x)
for key in recent_petition_dict:
    clean_array=[]
    for entry in recent_petition_dict[key][1]:
        if entry.select('p'):
            clean_array.append(entry.select('p'))
        else:
            clean_array=['NA']
        texts=[]
        if len(clean_array):
            for element in clean_array:
                soup=BeautifulSoup(str(element)).text
                texts.append(re.sub('(<p>)'," ",str(soup)))
    clean_recent_dict[key]=[ recent_petition_dict[key][0],texts,recent_petition_dict[key][2]]
            
        
 
    

In [389]:
pk.dump(clean_recent_dict,open('recent_petition0609.pkl','wb'))

In [ ]:
vic_petition_dict={}
ua = UserAgent()
user_agents=[ua.ie, ua.msie,
              ua['Internet Explorer'], ua.opera,
              ua.chrome,ua.google,
              ua['google chrome'],
              ua.firefox,ua.ff,
              ua.safari,ua.random]
#for i in xrange(1,20):
with open('victories_final.pkl','rb') as f:
    url_chuck=pk.load(f)
    #petition_chuck={}
for url in url_chuck.values():
    one_petition=get_petition_data_recent(user_agents,str(url))
    sleep(random.randint(1,10))
    petition_chuck.update(one_petition)
    vic_petition_dict.update(one_petition)
    #pk.dump(petition_chunck,open('recent_petition_chunk'+str(i)+'.pkl','wb'))
#pk.dump(vic_petition_dict,open('vic_petition.pkl','wb'))

In [371]:
clean_vic_dict={}
remove_fish=lambda x: re.sub('(<.*?>)'," ",x)
for key in vic_petition_dict:
    clean_array=[]
    for entry in vic_petition_dict[key][1]:
        if entry.select('p'):
            clean_array.append(entry.select('p'))
        else:
            clean_array=['NA']
        texts=[]
        if len(clean_array):
            for element in clean_array:
                soup=BeautifulSoup(str(element)).text
                texts.append(re.sub('(<p>)'," ",str(soup)))
    clean_vic_dict[key]=[vic_petition_dict[key][0],texts,vic_petition_dict[key][2]]
            
        
 
    
    

In [367]:
clean_vic_dict={}
remove_fish=lambda x: re.sub('(<p>)'," ",x)
for key in vic_petition_dict:
    clean_array=[]
    for entry in vic_petition_dict[key][1]:
        clean_array.append(remove_fish(str(entry)))
         
    clean_vic_dict[key]=[vic_petition_dict[key][0],texts,vic_petition_dict[key][2]]
        

In [373]:
len(clean_vic_dict)
pk.dump(clean_vic_dict,open('clean_vic_petition.pkl','wb'))